In [1]:
import pandas as pd
import datetime
import calendar

#### Process data from file

In [2]:
def read_data(filename):
    return pd.read_excel(filename)

In [3]:
def generate_from(start, end, interval=3600):
    interval_num = int((end-start).total_seconds()/interval)
    return pd.DataFrame({'From Time': [start+datetime.timedelta(seconds=i*interval) for i in range(interval_num)]})

In [4]:
def generate_fields(data):
    data['date'] = data['active'].apply(lambda x: datetime.datetime.date(x))
    data['time'] = data['active'].apply(lambda x: datetime.datetime.time(x))
    data['day of week'] = data['date'].apply(lambda x: calendar.day_name[x.weekday()])
    data['from'] = data['active'].apply(lambda x: x.replace(minute=0, second=0))
    data['to'] = data['from'].apply(lambda x: x + datetime.timedelta(hours=1))
    return data

In [5]:
def generate_from(start, end, interval=3600):
    interval_num = int((end-start).total_seconds()/interval)
    return pd.DataFrame({'from': [start+datetime.timedelta(seconds=i*interval) for i in range(interval_num)]})

def generate_to(start, end, interval=3600):
    interval_num = int((end-start).total_seconds()/interval)
    return pd.DataFrame({'to': [start+datetime.timedelta(seconds=(i+1)*interval) for i in range(interval_num)]})

In [6]:
def generate_time_table(data, interval=3600):
    start = min(data['from'])
    end = max(data['to'])
    interval_num = int((end-start).total_seconds()/interval)
    from_time = pd.DataFrame({'from': [start+datetime.timedelta(seconds=i*interval) for i in range(interval_num)]})
    to_time =  pd.DataFrame({'to': [start+datetime.timedelta(seconds=(i+1)*interval) for i in range(interval_num)]})
    time_table = pd.concat([from_time, to_time], axis=1)
    time_table['date'] = time_table['from'].apply(lambda x: datetime.datetime.date(x))
    time_table['time'] = time_table['from'].apply(lambda x: datetime.datetime.time(x))
    time_table['day of week'] = time_table['date'].apply(lambda x: calendar.day_name[x.weekday()])
    return time_table

In [7]:
def generate_gap(time_template, time):
    temp_list = list(time['from'])
    time_template['gap'] = time_template['from'].apply(lambda x: False if x in temp_list else True)
    return time_template

In [8]:
def generate_activities_count(data, all_flag=True, daily=True):
    if all_flag:
        if daily:
            activities = pd.DataFrame(data.groupby(['date', 'day of week']).size())
            activities.columns = ['all daily count']
        else:
            activities = pd.DataFrame(data.groupby(['from']).size())
            activities.columns = ['all hourly count']
    else:
        if daily:
            activities = pd.DataFrame(data.groupby(['name', 'date', 'day of week']).size())
            activities.columns=['ind daily count']
        else:
            activities = pd.DataFrame(data.groupby(['name', 'from']).size())
            activities.columns = ['all hourly count']
    return activities

In [9]:
def run_data_processing(filename, write_data=False):
    data = read_data(filename)
    data = generate_fields(data)
    hour_table = generate_time_table(data)
    hour_table = generate_gap(hour_table, data)
    date_table = generate_time_table(data, interval=3600*24)
    all_daily_activities = generate_activities_count(data, all_flag=True, daily=True)
    ind_daily_activities = generate_activities_count(data, all_flag=False, daily=True)
    all_hourly_activities = generate_activities_count(data, all_flag=True, daily=False)
    ind_hourly_activities = generate_activities_count(data, all_flag=False, daily=False)

    if write_data:
        with pd.ExcelWriter('processed_data.xlsx') as writer:
            data.to_excel(writer, sheet_name='data', index=False)
            hour_table.to_excel(writer, sheet_name='hour_table', index=False)
            date_table.to_excel(writer, sheet_name='date_table', index=False)
            all_daily_activities.to_excel(writer, sheet_name='all_daily_activities')
            ind_daily_activities.to_excel(writer, sheet_name='ind_daily_activities')
            all_hourly_activities.to_excel(writer, sheet_name='all_hourly_activities')
            ind_hourly_activities.to_excel(writer, sheet_name='ind_hourly_activities') 

In [10]:
# start_input = "9/1/2020 12:00 AM"
# end_input = "10/1/2020 12:00 AM"
# format = '%m/%d/%Y %I:%M %p'
# start = datetime.datetime.strptime(start_input, format)
# end = datetime.datetime.strptime(end_input, format)

#### Test Script

In [11]:
filename = 'raw_data.xlsx'
run_data_processing(filename, write_data=True)

In [12]:
# data = read_data(filename)
# data = generate_fields(data)
# hour_table = generate_time_table(data)
# hour_table = generate_gap(hour_table, data)
# date_table = generate_time_table(data, interval=3600*24)
# all_daily_activities = generate_activities_count(data, all_flag=True)
# ind_daily_activities = generate_activities_count(data, all_flag=False)